In [1]:
import pandas as pd
import requests
from openai import OpenAI
from pydantic import BaseModel
import pandas as pd
from tqdm import tqdm
import concurrent.futures
import json

# Initialize OpenAI client
client = OpenAI()

# Define response models using Pydantic for book validation

class Cafe(BaseModel):
    name: str
    address: str
    rating: float
    less_formal_name: str

# Function to call OpenAI for book extraction
def extract_cafe_address(row):
    prompt = (
        f"Find me the nearest coffee shop to this address: {row['match']}. Make sure the rating is above 4.0, avoid coffee chains like Dunkin, Starbucks, etc. In addition to name and address, return a less formal name of saying the name. For example, if a coffee is called The Ground Coffee Establishment, just say the Ground."
    )
    
    try:
        response = client.beta.chat.completions.parse(
            model="gpt-4o-mini",
            messages=[{"role": "user", "content": prompt}],
            response_format=Cafe
        )
        return response.choices[0].message.parsed
    except Exception as e:
        return [{"error": str(e)}]

df = pd.read_csv('filtered_people.csv')

tqdm.pandas()

df["cafe_recommendation"] = df.progress_apply(extract_cafe_address, axis=1)







100%|██████████| 281/281 [06:53<00:00,  1.47s/it]


In [2]:
df.to_csv('cofee_locations.csv')